In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
folder = '../../data/processed_data'
items_set=pickle.load(open(folder + '/item_set','rb'))
bundle_item_map=pickle.load(open(folder + '/bundle_item_map','rb'))
user_bundle_map=pickle.load(open(folder + '/user_bundle_map','rb'))
user_item_map=pickle.load(open(folder + '/user_item_map','rb'))
bundle_diversity_map=pickle.load(open(folder + '/bundle_diversity_map','rb'))
item_data=pickle.load(open(folder + '/all_items','rb'))
itemID_appID_map = pickle.load(open(folder + '/item_id_lookup','rb'))

In [40]:
bundle_final_price_map = pickle.load(open(folder + '/bundle_final_price_map','rb'))
bundle_discount_map = pickle.load(open(folder + '/bundle_discount_map','rb'))
bundle_price_map = pickle.load(open(folder + '/bundle_price_map','rb'))
item_price_map = pickle.load(open(folder + '/item_price_map','rb'))
item_id_lookup = pickle.load(open(folder + '/item_id_lookup','rb'))

In [3]:
appID_item_map = {}
for date in item_data:
    appID = int(date['appid'])
    appID_item_map[appID] = date

In [4]:
with open(folder + '/appID_item_map.pickle', 'wb') as handle:
    pickle.dump(appID_item_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
itemID_item_map = {}
for itemID in itemID_appID_map:
    appID = itemID_appID_map[itemID]
    if appID in appID_item_map:
        itemID_item_map[itemID] = appID_item_map[appID]
    else:
        itemID_item_map[itemID] = {}

In [6]:
with open(folder + '/itemID_item_map', 'wb') as handle:
    pickle.dump(itemID_item_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [55]:
for i in range(len(itemID_item_map)):
    try:
        if int(itemID_item_map[i]['appid']) != itemID_appID_map[i]:
            print int(itemID_item_map[i]['appid']),  itemID_appID_map[i]
    except:
        i = i

In [8]:
itemID_appID_map[2]

516100

In [56]:
users = []
for user in user_item_map:
    for item in user_item_map[user]:
        users += [user, item],
user_item = pd.DataFrame(users,  columns=['userID', 'itemID'])

In [57]:
 itemMap = pd.DataFrame([ [x] + [d[k] for k in d] for x, d in itemID_item_map.items()], 
                        columns=['itemID', 'original_price', 'discount_original_price', 'discount_final_price',
                                'tags', 'reviews', 'appid', 'date'])

In [58]:
user_items = pd.merge(user_item, itemMap, how='left', on = 'itemID')

In [14]:
user_items.to_csv(folder + '/user_items_matrix', index=False)

In [74]:
def convert2Float(x):
    if isinstance(x, float):
        return -1
    else:
        idx = x.find('$')
        if idx == -1:
            return 0
        else:
            return float((x[idx+1:]))

In [87]:
itemID_price_map = {}
itemID_discount_map = {}
orign, discount = 0, 0 
for itemID in itemID_item_map:
    item = itemID_item_map[itemID]
    if 'discount_final_price' not in item:
        itemID_price_map[itemID] = -2
        itemID_discount_map[itemID] = 0
#         orign += 1
#         discount += 1
    elif item['discount_final_price'] == '' and item['original_price'] == '':
        itemID_price_map[itemID] = -3
        itemID_discount_map[itemID] = 0
    elif item['original_price'] == '' and item['discount_original_price'] != '' and\
        item['discount_final_price'] != '':
        p2 = convert2Float(item['discount_original_price'])
        p3 = convert2Float(item['discount_final_price'])
        p1 = p2 + p3
        itemID_discount_map[itemID] =  1 - (float(p3) / p1)
    elif item['discount_final_price'] == '' or item['original_price'] == '':
        if item['discount_final_price'] == '':
            itemID_price_map[itemID] = convert2Float(item['original_price'])
            orign += 1
        else:
            itemID_price_map[itemID] = convert2Float(item['discount_final_price'])
            discount += 1
        itemID_discount_map[itemID] = 0
    else:
        p3 = convert2Float(item['discount_final_price'])
        p1 = convert2Float(item['original_price'])
        itemID_price_map[itemID] = int(p1 + p3 / 2)
        if p3 < p1:
            itemID_discount_map[itemID] =   1 - (float(p3) / p1)
        else:
            itemID_discount_map[itemID] = 0

In [77]:
count1, count2, count3, count13, count123 = 0, 0, 0, 0, 0
for itemID in itemID_item_map:
    item = itemID_item_map[itemID]
    if 'original_price' not in item: continue
    if(item['original_price'] == ''):
        count1 += 1
    if(item['discount_original_price'] == ''):
        count2 += 1
    if(item['discount_final_price'] == ''):
        count3 += 1
    if (item['original_price'] == '') and (item['discount_final_price'] == ''):
        count13 += 1
    if item['original_price'] == '' and item['discount_original_price'] != '' and item['discount_final_price'] != '':
#         print item['discount_original_price'], item['discount_final_price']
        count123 += 1

In [69]:
print count1, count2, count3, count13, count123

105 2026 17 0 101


In [86]:
print orign, discount
print len(itemID_discount_map)
for k in itemID_discount_map:
    if itemID_discount_map[k] < 1:
        print itemID_discount_map[k]
        break

17 4
2819
0.90991902834


In [88]:
with open(folder + '/itemID_discount_map', 'wb') as handle:
    pickle.dump(itemID_discount_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
list = []
for id in bundle_final_price_map:
    d = (1 - float(bundle_final_price_map[id]) / bundle_price_map[id]) * 100
    list += [d, bundle_discount_map[id]],

In [28]:
for x in list:
    if abs(x[0] - x[1]) > 1:
        print x

In [30]:
print len(itemID_price_map)
print len(list), len(itemID_discount_map)
#321 409

2819
615 2819


In [16]:
with open(folder + '/itemID_price_map', 'wb') as handle:
    pickle.dump(itemID_price_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
itemID_item_map[0]

{'appid': '409600',
 'date': u'Dec 11, 2015',
 'discount_final_price': u'$15.17',
 'discount_original_price': '',
 'original_price': u'$5.99',
 'reviews': [u'Mixed'],
 'tags': [u'Adventure', u'Casual', u'Hidden Object', u'Point & Click']}

In [ ]:
user_price_mean = user_items.groupby('userID')['price'].apply(lambda x: np.mean(x.tolist())).to_dict()
user_price_std = user_items.groupby('userID')['price'].apply(lambda x: np.std(x.tolist())).to_dict()

In [ ]:
for k in user_price_std:
    if user_price_std[k] == 0:
        print k, user_price_mean[k], user_price_std[k]
        user_price_std[k] = 500

In [ ]:
with open(folder + '/user_price_std', 'wb') as handle:
    pickle.dump(user_price_std, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [37]:
for k in item_price_map:
    print k, item_price_map[k]
    break

0 4.79


In [66]:
print len(item_price_map)

2819


In [44]:
print itemID_item_map[1]
print item_price_map[1]

{'original_price': u'$5.99', 'discount_original_price': '', 'discount_final_price': '', 'tags': [u'Strategy', u'Medieval', u'Classic', u'RTS', u'Base-Building', u'Simulation', u'Economy', u'Multiplayer', u'City Builder', u'Resource Management', u'Singleplayer', u'Great Soundtrack', u'Historical', u'Atmospheric', u'2D'], 'reviews': [u'Very Positive', u'Very Positive'], 'appid': '40960', 'date': u'Oct 19, 2001'}
19.99


In [46]:
print appID_item_map[270910]

{'original_price': u'$14.99', 'discount_original_price': '', 'discount_final_price': '', 'tags': [u'Strategy', u'Action', u'Indie', u'Multiplayer', u'Turn-Based', u'Funny'], 'reviews': [u'Mixed'], 'appid': '270910', 'date': u'Jul 16, 2015'}


In [47]:
print item_id_lookup[1], len(item_id_lookup)

40960 2819


In [48]:
dict = {}
for key in item_id_lookup:
    val = item_id_lookup[key]
    dict[val] = key

In [52]:
print len(dict)

2819


In [65]:
print len(user_items.userID.unique())

29634


In [ ]:
print len(user_items.userID.unique())